<a href="https://colab.research.google.com/github/vitaldb/planb/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction Model for Hepatocellular cardinoma in Hepatitis B patients

## Upgrade XGboost library

In [6]:
!pip uninstall xgboost -y
!pip install xgboost

Found existing installation: xgboost 1.6.2
Uninstalling xgboost-1.6.2:
  Successfully uninstalled xgboost-1.6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xgboost-1.6.2-py3-none-manylinux2014_x86_64.whl (255.9 MB)


## Download model and sample files

In [15]:
!wget https://github.com/vitaldb/hbvhcc/raw/main/model.json
!wget https://github.com/vitaldb/hbvhcc/raw/main/sample.csv

--2022-11-24 15:33:10--  https://github.com/vitaldb/hbvhcc/raw/main/model.json
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/vitaldb/planb/raw/main/model.json [following]
--2022-11-24 15:33:10--  https://github.com/vitaldb/planb/raw/main/model.json
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitaldb/planb/main/model.json [following]
--2022-11-24 15:33:10--  https://raw.githubusercontent.com/vitaldb/planb/main/model.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60814 (59K) [text/plain]
Saving 

## Sample code for single patient



In [16]:
import xgboost as xgb
import numpy as np

# sample values
male = 0
age = 48
e_t = 1
lc = 1  # None for missing value
plt = 172
alb = 4.1
tbil = 1.1
alt = 122
dna = 74279943
hbeag = 1

model = xgb.Booster()
model.load_model('model.json')
x = np.array([male, age, e_t, lc, plt, alb, tbil, alt, dna, hbeag]).astype(float)
x = x[None, ...]
x = xgb.DMatrix(x)
y = model.predict(x)[0]

print('score = {:.3f}'.format(y))

score = 0.196


## Sample code using CSV file for multiple patients 

In [17]:
import xgboost as xgb
import numpy as np
import pandas as pd

model = xgb.Booster()
model.load_model('model.json')

df = pd.read_csv('sample.csv')
x = df[['male', 'age', 'e_t', 'lc', 'plt', 'alb', 'tbil', 'alt', 'dna', 'hbeag']].astype(float).values
df['pred'] = model.predict(xgb.DMatrix(x))
df.to_csv('prediction.csv')
print(df)

   no  male  age  e_t  lc  plt  alb  tbil  alt        dna  hbeag      pred
0   1     1   33    0   0  167  4.4   1.1   84  170000000      1  0.095564
1   2     0   56    0   0  168  3.6   0.7  223    7990000      1  0.163563
2   3     0   62    1   1  164  4.3   0.5  104       6080      0  0.286515
3   4     1   46    1   0  317  4.3   0.7   66      45100      0  0.095078
4   5     1   54    1   1  122  4.6   1.3  147   12500000      0  0.412359
5   6     1   30    0   0  198  4.4   1.4  955  170000000      1  0.057932
6   7     1   48    0   1  111  4.2   1.3   95     262000      0  0.630031
7   8     0   58    1   1  138  4.3   0.7   45      17100      0  0.355294
8   9     0   50    0   1  154  4.2   1.0   61     790000      0  0.450027
9  10     0   65    0   0  187  4.6   0.6   23          0      0  0.201603
